<a href="https://colab.research.google.com/github/dvschultz/stylegan2-training/blob/main/SG2_ADA_Generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#StyleGAN2 Manipulation
#### Inference, Truncation, Interpolation, Mixing, and Projection

This notebook covers manipulating a StyleGAN2-ADA model to produce images and videos. It was made by [Derrick Schultz](https://twitter.com/dvsch) and [Lia Coleman](https://twitter.com/Lialialiacole) for their StyleGAN2 courses.

Parts of this notebook contain ideas and code from [Mikael Christensen](https://colab.research.google.com/drive/1ShgW6wohEFQtqs_znMna3dzrcVoABKIH) and [Dan Shiffman](https://www.youtube.com/watch?v=vEetoBuHj8g).

In [ ]:
%tensorflow_version 1.x
!nvidia-smi

# Setup: Download the StyleGAN2 repo 

This will install all the necessary libraries to use the StyleGAN2 repo. Press the play button or `shift+return` to run each cell.

Only run the next cell once per session.


In [ ]:
!git clone https://github.com/dvschultz/stylegan2-ada
!pip install opensimplex

In [ ]:
%cd stylegan2-ada

### Download your trained model from Google Drive
Input the google id of your trained .pkl file. 
The -O option here is to specify the output file name and location.

In [ ]:
!gdown --id 11jjGGqKJs9KpqAgKkRQde15vTjfrKOUO -O /content/network.pkl

In [ ]:
!python generate.py --help

#Generate Images

Also known as "Inference", "Evaluation" or "Testing" the model.
The following command will generate 50 sample images from the model.

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

In [ ]:
!python generate.py generate-images --network="/content/network.pkl" --seeds=0-10 --outdir="./out/"

#### With conditional model

In [ ]:
!python generate.py generate-images --network="/content/network.pkl" --seeds=0-10 --outdir="./out/". --class=1

# Truncation Traversal


Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. Most people choose between 0.5 and 1.0, but technically its infinite. 

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

###Options 
`--network`: Again, point this to your .pkl file.

`--seed`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)

In [ ]:
!python generate.py truncation-traversal --network="/content/network.pkl" --seed=1 --start=-2.0 --stop=2.0 --increment=0.25 --outdir="./tt" --fps=30

# Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at two different examples of interpolation: a linear interpolation and a random noise loop.

Both methods require the following options:

`--network`

`--walk-type`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`


### Linear Interpolation

Linear interpolation generates a linear path from one seed to another. The makers of StyleGAN say that doing this in the w space produces the best disentangled interpolations. But let’s start by looking at it in z space.

`--seeds`: Use images you generated to control the interpolation points. If your first and last seed are the same this will produce a loop (nice for Instagram and gifs!)

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="line-z" --seeds=0,2,5,0 --outdir="./z-walk"


Next let’s look at linear interpolation in w space. To do this we set `--walk-type` to `line-w`.

I recommend using the exact same seeds so you see the difference. It’s often very subtle but it is different.


In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="line-w" --seeds=0,2,5,0 --outdir="./w-walk"

### Noise Loop Interpolation

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

Seeds aren’t used here but you do need to set one for 🌟reasons🌟. Just leave it at zero but know that changing this value won’t do much.

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--start_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

In [ ]:
!python generate.py generate-latent-walk --network="/content/network.pkl" --walk-type="noiseloop" --start_seed=0 --outdir="./noise1"

# Near Neighbors

**Added by popular demand**

Let’s say you have a seed you like, but want to see other images like it to see if there’s something better. Now you can with the `near-neighbor` argument.

### Options
`--network`, `--seeds`, and `--truncation_psi` work the same as above.

`--diameter`: this sets how far away from the seed you want to generate images. `.0000001` is really close, `.5` is reallly far.

`--num_samples`: how many samples you want to produce

`--save_vector`: this will save the vector as a file in the .npy format. You can then use this for interpolation (not super well supported right now, but can be used manually—see the Projection code for an example of reading a .npy file and interpolating it).




In [ ]:
!python generate.py generate-neighbors --network="/content/network.pkl" --seeds=9 --outdir="./neighbors"

# Style Mixing

Since seeds are just points in a vector space, we can do math things to them, like adding them together. You could do this thru linear interpolation and using the middle frame, but if you want to visualize a number of options here’s a simple script to do it. This takes a number of seeds to produce a grid showing what happens when you add the row and column together (this will make more sense after running it).

In [ ]:
!python style_mixing.py --outdir=out --rows=85,100,75,458,1500 --cols=55,821,1789,293 --network="/content/network.pkl"

# Flesh Digressions

In [ ]:
!python aydao_flesh_digressions.py --pkl "/content/network.pkl" --psi=0.6 --radius_small=10.0 --radius_large=800.00

# Projection

Projection is the process of taking an image from outside the model and finding its nearest representation inside the model.

For projection, we’re gonna do something a little different in this notebook and load the code directly into Colab. Don’t worry if this looks scary—there’s only two things you want to edit in the code for any projection.

In [ ]:
!python projector.py --outdir=out2 --target=/content/redbull.png --network=https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada/pretrained/ffhq.pkl